In [ ]:
# Setup the library
import pyrogue as pr
import os, sys
import matplotlib.pyplot as plt
import time
import numpy as np
import math

top_level=f'{os.getcwd()}/../../'
pr.addLibraryPath( f'{top_level}/scripts' )
import setupLibPaths
import epix_hr_single_10k
import ePixViewer.Cameras as cameras


In [ ]:
from importlib import reload
reload(cameras)

MAX_NUMBER_OF_FRAMES_PER_BATCH  = 1000
cameraType            = 'ePixHr10kTBatcher'
bitMask               = 0xffff
currentCam = cameras.Camera(cameraType = cameraType)
currentCam.bitMask = bitMask
currentCam.MAX_NUMBER_OF_FRAMES_PER_BATCH  = MAX_NUMBER_OF_FRAMES_PER_BATCH

initSequence = 4


# Helper functions

In [ ]:
def read_asic_test():
    print(filename)
    ########################################################################################################################3
    f = open(filename, mode = 'rb')
    imgDesc = []
    for i in range(1):
        print("Starting to get data set %d" % (i))
        allFrames = currentCam.getData(f,8)
        imgDesc2 = currentCam.getDescImaData(allFrames[:,:-8])
        if i == 0:
            headers = allFrames[:,0:6]
            imgDesc = imgDesc2
        if i == 1: #skips the first dataset
            headers = allFrames[:,0:6]
            imgDesc = imgDesc2
        else:
            headers = np.concatenate((headers, allFrames[:,0:6]),0)
            imgDesc = np.concatenate((imgDesc, imgDesc2),0)
        if allFrames.shape[0] != MAX_NUMBER_OF_FRAMES_PER_BATCH:
            break


    numberOfFrames = allFrames.shape[0]
    print(allFrames.shape)
    ###########################################################################################################################
    return imgDesc


In [ ]:
def noise_print(offset = 0, width = 384):
    print(imgDesc[100].shape)
    imgDescLoc = imgDesc[:,:,offset:offset+width]
    hg_n  = imgDescLoc.astype('float64')
    hg_n -= np.median(hg_n, axis=0, keepdims=True)
    hg_n -= np.median(hg_n, axis=(1,2), keepdims=True)
    #hg_n -= np.median(hg_n, axis=2, keepdims=True) # rows (cols)
    #hg_n -= np.median(hg_n, axis=1, keepdims=True) # cols (rows)
    imgDescCMC = hg_n
    
    darkAvg  = np.mean(imgDescLoc,0)
    noiseMap = np.std(imgDescLoc,0)
    medianNoise = np.median(noiseMap)
    #noiseMap[noiseMap>medianNoise*4]=0
    
    darkAvgCMC  = np.mean(imgDescCMC,0)
    noiseMapCMC = np.std(imgDescCMC,0)
    medianNoiseCMC = np.median(noiseMapCMC)
    noiseMapCMC[noiseMapCMC>medianNoiseCMC*5]=0
    
    plt.figure(1,figsize=(12,8),dpi=150)
    plt.imshow(noiseMap, cmap='gray', vmin=-10, vmax=50)
    plt.colorbar(location='right', anchor=(0, 0.3), shrink=0.7)

    plt.figure(2,figsize=(12,8),dpi=150)
    dataset = noiseMap
    plt.hist(dataset.reshape(-1), bins=100, label=f'\nNoise Map distribution AHL, Avg. Dark level %d, Mediam Noise %d' % (np.mean(np.mean(darkAvg)), medianNoise)) 
    dataset = noiseMapCMC
    plt.hist(dataset.reshape(-1), bins=100, label=f'\nNoise Map distribution AHL, Avg. Dark level %d, Mediam Noise %d' % (np.mean(np.mean(darkAvgCMC)), medianNoiseCMC)) 
    plt.yscale('log')
    plt.title("histogram") 
    plt.legend()
    plt.show()
          
    plt.figure(3,figsize=(12,8),dpi=150)
    print(imgDesc.shape)
    dataset = imgDesc[:,:,offset:offset+192]
    print(dataset.shape)
    plt.plot(dataset[:,10,:]) 
    #plt.legend()
    
    plt.figure(4,figsize=(12,8),dpi=150)
    print(imgDesc.shape)
    dataset = imgDesc[:,:,offset:offset+192]
    print(dataset.shape)
    plt.plot(dataset[:,10,86])
    #plt.legend()
    
    return noiseMap, darkAvg

# Pixel Optimization

In [ ]:
PreampList = [7, 6, 5, 4, 3, 2, 1, 0]
Vld1_b = 0
Pixel_CB= 7
Filter_DAC= 17
VRef_DAC = 58

for Preamp in PreampList:
   
    ######################################################################################################################
    # Remove previous dataset
    filename = (f'/u2/ddoering/ASIC_V4_ASIC2_SB/ePixHR10kTLCLSIITiming_ASICV4_ASIC2_noise_320MHz_initSequence_%d_Preamp_%d_Vld1b_%d_PixelCB_%d_Filter_DAC_%d_VRrefdac_%d_run1.dat' % (initSequence, Preamp, Vld1_b, Pixel_CB, Filter_DAC, VRef_DAC))
    ########################################################################################################################
    imgDesc = read_asic_test()
    ########################################################################################################################
    [noiseMap, darkAvg] = noise_print(0, 192)

In [ ]:
Preamp = 7
Vld1_bList = [0,1,2,3]
Pixel_CB= 7
Filter_DAC= 17
VRef_DAC = 58

for Vld1_b in Vld1_bList:
   
    ######################################################################################################################
    # Remove previous dataset
    filename = (f'/u2/ddoering/ASIC_V4_ASIC2_SB/ePixHR10kTLCLSIITiming_ASICV4_ASIC2_noise_320MHz_initSequence_%d_Preamp_%d_Vld1b_%d_PixelCB_%d_Filter_DAC_%d_VRrefdac_%d_run1.dat' % (initSequence, Preamp, Vld1_b, Pixel_CB, Filter_DAC, VRef_DAC))
    ########################################################################################################################
    imgDesc = read_asic_test()
    ########################################################################################################################
    [noiseMap, darkAvg] = noise_print(0)

In [ ]:
Preamp = 7
Vld1_b = 0
Pixel_CBList= [7,6,5,4,3,2,1,0]
Filter_DAC= 17
VRef_DAC = 58

for Pixel_CB in Pixel_CBList:
   
    ######################################################################################################################
    # Remove previous dataset
    filename = (f'/u2/ddoering/ASIC_V4_ASIC2_SB/ePixHR10kTLCLSIITiming_ASICV4_ASIC2_noise_320MHz_initSequence_%d_Preamp_%d_Vld1b_%d_PixelCB_%d_Filter_DAC_%d_VRrefdac_%d_run1.dat' % (initSequence, Preamp, Vld1_b, Pixel_CB, Filter_DAC, VRef_DAC))
    ########################################################################################################################
    imgDesc = read_asic_test()
    ########################################################################################################################
    [noiseMap, darkAvg] = noise_print(0)

In [ ]:
Preamp = 7
Vld1_b = 0
Pixel_CB= 7
Filter_DACList=[7, 13, 15, 17, 19, 21, 27, 37, 47, 57]
VRef_DAC = 58

for Filter_DAC in Filter_DACList:
   
    ######################################################################################################################
    # Remove previous dataset
    filename = (f'/u2/ddoering/ASIC_V4_ASIC2_SB/ePixHR10kTLCLSIITiming_ASICV4_ASIC2_noise_320MHz_initSequence_%d_Preamp_%d_Vld1b_%d_PixelCB_%d_Filter_DAC_%d_VRrefdac_%d_run1.dat' % (initSequence, Preamp, Vld1_b, Pixel_CB, Filter_DAC, VRef_DAC))
    ########################################################################################################################
    imgDesc = read_asic_test()
    ########################################################################################################################
    [noiseMap, darkAvg] = noise_print(0)

In [ ]:
#Makes sure triggers are off

Preamp = 7
Vld1_b = 0
Pixel_CB= 7
Filter_DAC=17
VRef_DACList = [3, 13, 23, 33, 43, 48, 53, 58, 63]

for VRef_DAC in VRef_DACList:

    ######################################################################################################################
    # Remove previous dataset
    filename = (f'/u2/ddoering/ASIC_V4_ASIC2_SB/ePixHR10kTLCLSIITiming_ASICV4_ASIC2_noise_320MHz_initSequence_%d_Preamp_%d_Vld1b_%d_PixelCB_%d_Filter_DAC_%d_VRrefdac_%d_run1.dat' % (initSequence, Preamp, Vld1_b, Pixel_CB, Filter_DAC, VRef_DAC))
    ########################################################################################################################
    imgDesc = read_asic_test()
    ########################################################################################################################
    [noiseMap, darkAvg] = noise_print(0)